In [ ]:
!pip install gradio
!pip install transformers
!pip install typing_extensions
!pip install transformers[torch]
!pip install -U gradio
!pip install transformers --upgrade
!pip install torch torchvision pytorch-lightning

In [ ]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from transformers import TextDataset, DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments

# Load pre-trained GPT-2 model and tokenizer
model_name = "gpt2"
model = GPT2LMHeadModel.from_pretrained(model_name)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

# Load your preprocessed data
with open("normans_wikipedia.txt", "r", encoding="utf-8") as file:
    data = file.read()


# Specify the output directory
output_dir = "/content/normans_fine-tuned"
os.makedirs(output_dir, exist_ok=True)


# Tokenize and encode the data
input_ids = tokenizer.encode(data, return_tensors="pt")

# Create a dataset and data collator
dataset = TextDataset(
    tokenizer=tokenizer,
    file_path="normans_wikipedia.txt",
    block_size=128,
)
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

# Fine-tune the model
training_args = TrainingArguments(
    output_dir="/content/normans_fine-tuned",
    overwrite_output_dir=True,
    num_train_epochs=45,
    per_device_train_batch_size=2,
    save_steps=10_000,
    save_total_limit=2,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
)

# Training loop
try:
    for step, inputs in enumerate(dataset):
        loss = trainer.train_step(model, inputs)

        # Print the loss at each step
        if step % training_args.logging_steps == 0:
            print(f"Step {step}, Training Loss: {loss:.4f}")

except KeyboardInterrupt:
    print("Training interrupted by user.")

# Access the final training loss
final_training_loss = loss
print(f"Final Training Loss: {final_training_loss}")

# Save the fine-tuned model
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

# Load the fine-tuned model
fine_tuned_model = GPT2LMHeadModel.from_pretrained(output_dir)


Token indices sequence length is longer than the specified maximum sequence length for this model (8762 > 1024). Running this sequence through the model will result in indexing errors
/usr/local/lib/python3.10/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


Step,Training Loss
500,1.756700
1000,0.182600
1500,0.057700


TrainOutput(global_step=1530, training_loss=0.6535929464826397, metrics={'train_runtime': 188.1343, 'train_samples_per_second': 16.265, 'train_steps_per_second': 8.132, 'total_flos': 199888404480000.0, 'train_loss': 0.6535929464826397, 'epoch': 45.0})

In [ ]:
import gradio as gr
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import torch
import os

# Load pre-trained GPT-2 model and tokenizer
model_name = "gpt2"
model = GPT2LMHeadModel.from_pretrained(model_name)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

# Load the fine-tuned model
output_dir = "/content/normans_fine-tuned"
fine_tuned_model = GPT2LMHeadModel.from_pretrained(output_dir)

# Function to generate responses from the fine-tuned model
def generate_response(user_input):
    # Tokenize and encode user input
    user_input_ids = tokenizer.encode(user_input, return_tensors="pt")

    # Generate response from the fine-tuned model
    generated_output = fine_tuned_model.generate(
        user_input_ids,
        max_length=100,
        num_beams=5,
        no_repeat_ngram_size=2,
        top_k=50,
        top_p=0.90,
        temperature=0.9
    )

    # Decode and return the generated response
    chatbot_response = tokenizer.decode(
        generated_output[0], skip_special_tokens=True)
    return "Chatbot: " + chatbot_response

# Create a Gradio interface
iface = gr.Interface(
    fn=generate_response,
    inputs="text",
    outputs="text",
    live=True
)

# Launch the Gradio interface
iface.launch()
